In [1]:
# create datacard for 2 tag

import ROOT as rt
import csv
import re
import sys
import collections
import os
import math
from collections import OrderedDict
# import ot
import pandas as pd
import uproot
import scipy
import awkward
import numpy as np
import time
import numba
from numba import jit
from matplotlib import pyplot as plt
sys.path.append('/storage/af/user/christiw/gpu/christiw/llp/delayed_jet_analyzer/lib/')
sys.path.append('/storage/af/user/christiw/login-1/christiw/LLP/CMSSW_9_4_4/src/llp_analyzer/python/')
from helper import make_datacard_2sig, make_datacard_2tag, weight_calc
from histo_utilities import create_TH1D, create_TH2D, std_color_list, create_TGraph, make_ratio_plot
from helper_functions import deltaR, deltaPhi

import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()
CMS_lumi.writeExtraText = 0


print(sys.version)

Welcome to JupyROOT 6.18/02
3.6.8 (default, Aug  7 2019, 17:28:10) 
[GCC 4.8.5 20150623 (Red Hat 4.8.5-39)]


In [2]:
from platform import python_version
python_version()

'3.6.8'

# Load ntuples

In [ ]:
fpath =OrderedDict()
tree = OrderedDict()

scale = ['xiO_2p5_xiL_1', 'xiO_2p5_xiL_2p5', 'xiO_1_xiL_1']
prod = ['ggH']
portal = {
'higgs': [4, 5, 10, 15, 20],
'gluon': [3, 5, 10, 15, 20],
'vector':[2, 5, 10, 15, 20],
'photon':[2, 5, 10, 15, 20],
'darkphoton':[2, 5, 10, 15, 20],
}
# old_ctau = [500, 1000, 5000, 10000]
old_ctau = [10, 50, 100, 500, 1000, 5000, 10000, 50000, 100000]



ntupler_version = 'V1p17/'
analyzer_version = 'v2/v163/'


mc_path = {}
mc_path= '/storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/MC_all/'+analyzer_version+'/normalized/'
for x in scale:
    for p in portal.keys():
        for m in portal[p]:
            for ct in old_ctau:
#                 if not x == 'xiO_2p5_xiL_2p5':continue
#                 if not p == 'darkphoton':continue
#                 if not m == 5:continue
#                 if not ct == 500:continue
                key = 'HV_params_'+p+'_m_'+str(m) + '_ctau_'+str(ct)+'mm_'+x
                fpath[key] = mc_path+key+'.root'
                print(fpath[key])

# fpath['old']= '/storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p17//MC_Summer16/v1/v163//normalized/HV_params_darkphoton_m_5_ctau_500mm_xiO_2p5_xiL_2p5.root'
# fpath['new']= '/storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p17//MC_Summer16/v2/v163//normalized/HiddenValley_darkphoton_m_5_ctau_500_xiO_2p5_xiL_2p5.root'

data_path = '/storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p17/Data2018/v5/v170/normalized/'

# fpath['data'] = data_path + 'Run2_displacedJetMuonNtupler_V1p17_Data2016_Data2017_Data2018-HighMET_goodLumi.root'

NEvents = {}
NEvents_genweight = {}
for k,v in fpath.items():
    root_dir = uproot.open(v) 
    if not root_dir: 
        print(k, "zombie")
        continue
    tree[k] = root_dir['MuonSystem']
    NEvents[k] = root_dir['NEvents'][1]
    
    w = tree[k]["weight"].array()
    if not 'data' in k: 
        print(k, root_dir['NEvents']._fEntries)
        


root_dir = uproot.open('/storage/af/user/christiw/login-1/christiw/LLP/CMSSW_9_4_4/src/llp_analyzer/data/HiggsPtWeights/ZHToggZH_HiggsPtReweight.root') 
h_reweight = root_dir['higgsPthiggsEta']

# nCsc with different hit vetoing

In [6]:
NEVENTS = 2000000
XSEC = 48.58 #pb

In [11]:
%time

jetPt_cut = 50

weight = {}
nhits1 = {}
nhits2 = {}
sel_ev = {}
cond = {}
cond_dtnoise_2016 = {}
ggZH_weight = {}
higgsEta = {}
higgsPt = {}

nCosmic = {}
deltaEta = {}
deltaRCluster = {}
jetPt = {}
jetTightPassId = {}
runNum = {}
evtNum = {}
lumiNum = {}
sel_ev_dtnoise_2016 = {}
nhits1_dtnoise_2016 = {}
nhits2_dtnoise_2016 = {}
cluster_index = ''
addNoiseFlag = 1
# 0: 2 CSC; 1: 2DT; 2: csc+dt
category = 0


for k in list(tree.keys()):
#     if not 'ggH' in k:continue
    if   'data' in k:continue
########### SELECTION: CLUSTERS ############
#     print(k)
#     if not 'HV_params_higgs_m_10' in k and not 'data' in k:continue
    if 'data' in k: T = tree['data']
    else: T = tree[k]

    sel_csccluster = T.array('cscRechitCluster' + cluster_index + 'TimeSpreadWeightedAll')<20
    sel_csccluster = np.logical_and(sel_csccluster, np.abs(T.array('cscRechitCluster' + cluster_index + 'MetEENoise_dPhi'))<1.2)
    sel_csccluster = np.logical_and(sel_csccluster, T.array('cscRechitCluster' + cluster_index + 'JetVetoPt')<30)
    sel_csccluster = np.logical_and(sel_csccluster, T.array('cscRechitCluster' + cluster_index + 'Me11Ratio')<1)
    sel_csccluster = np.logical_and(sel_csccluster, np.logical_not(np.logical_and(T.array('cscRechitClusterMuonVetoPt') >= 30, T.array('cscRechitClusterMuonVetoGlobal'))))
    
    if 'data' in k: 
        sel_csccluster = np.logical_and(sel_csccluster, np.logical_and(T.array('cscRechitCluster' + cluster_index + 'TimeWeighted')< 12.5, \
                                                                         T.array('cscRechitCluster' + cluster_index + 'TimeWeighted') > -5))
    else: 
        sel_csccluster = np.logical_and(sel_csccluster, np.logical_and(T.array('cscRechitCluster' + cluster_index + 'TimeWeighted')+0.66 < 12.5, \
                                                                         T.array('cscRechitCluster' + cluster_index + 'TimeWeighted')+0.66 > -5))
        
    sel_dtcluster = np.abs(T.array('dtRechitClusterMetEENoise_dPhi')) < 1
    sel_dtcluster = np.logical_and(sel_dtcluster, np.logical_not(np.logical_and(T.array('dtRechitClusterMuonVetoPt') >= 10, T.array('dtRechitClusterMuonVetoLooseId'))))
    sel_dtcluster = np.logical_and(sel_dtcluster, np.abs(T.array('dtRechitClusterJetVetoPt')) < 50)
    sel_dtcluster = np.logical_and(sel_dtcluster, np.logical_not(np.logical_and(T.array('dtRechitClusterMaxStation')==1, T.array('dtRechitClusterMaxStationRatio')>0.9)))

    
    cut = 5
    station = (T.array('dtRechitClusterNSegmentStation1')>cut).astype(int)+(T.array('dtRechitClusterNSegmentStation2')>cut).astype(int)\
+(T.array('dtRechitClusterNSegmentStation3')>cut).astype(int)+(T.array('dtRechitClusterNSegmentStation4')>cut).astype(int)

    max_station = np.maximum(np.maximum(np.maximum(T.array('dtRechitClusterNSegmentStation1'), T.array('dtRechitClusterNSegmentStation2')), T.array('dtRechitClusterNSegmentStation3')), T.array('dtRechitClusterNSegmentStation4'))
    min_station = np.minimum(np.minimum(np.minimum(T.array('dtRechitClusterNSegmentStation1'), T.array('dtRechitClusterNSegmentStation2')), T.array('dtRechitClusterNSegmentStation3')), T.array('dtRechitClusterNSegmentStation4'))

    sel_dtcluster = np.logical_and(sel_dtcluster, np.logical_or(station<4, 1.0*min_station/max_station<0.4)) #remove if both clusters are 4 stations
    if addNoiseFlag and not 'data' in k: 
        sel_dtcluster = np.logical_and(sel_dtcluster, (T.array('dtRechitClusterSize')+T.array('dtRechitClusterNoiseHit')) >= 50)
    else: sel_dtcluster = np.logical_and(sel_dtcluster, T.array('dtRechitClusterSize') >= 50)



    if k in 'data': 
        sel_dtcluster = np.logical_and(sel_dtcluster, np.logical_not(T.array('dtRechitClusterNoiseVeto')))
        sel_dtcluster = np.logical_and(sel_dtcluster, np.logical_not(T.array('dtRechitClusterOverlap')))
    ###################### cosmic muon veto #############
    sel_cosmic = np.logical_and(T.array('dtRechitClusterNOppositeSegStation1')>0, T.array('dtRechitClusterNOppositeSegStation2')>0)
    sel_cosmic = np.logical_and(sel_cosmic, T.array('dtRechitClusterNOppositeSegStation3')>0)
    sel_cosmic = np.logical_and(sel_cosmic, T.array('dtRechitClusterNOppositeSegStation4')>0)
    sel_cosmic = np.logical_and(sel_cosmic, T.array('dtRechitClusterNOppositeSegStation1')+T.array('dtRechitClusterNOppositeSegStation2')+\
                               T.array('dtRechitClusterNOppositeSegStation3')+T.array('dtRechitClusterNOppositeSegStation4')>=6)
    nstation = (T.array('dtRechitClusterNSegmentStation1')>1).astype(int)+(T.array('dtRechitClusterNSegmentStation2')>1).astype(int)\
    +(T.array('dtRechitClusterNSegmentStation3')>1).astype(int)+(T.array('dtRechitClusterNSegmentStation4')>1).astype(int)
    
    sel_dtcluster = np.logical_and(sel_dtcluster, np.logical_not(np.logical_and(nstation>=3, sel_cosmic)))

    noise_2016 = np.logical_and(T.array('dtRechitClusterMaxStation') == 2, T.array('dtRechitClusterWheel') == 1)
    noise_2016 = np.logical_and(noise_2016, T.array('dtRechitClusterPhi') > math.pi/12)
    noise_2016 = np.logical_and(noise_2016, T.array('dtRechitClusterPhi') < math.pi/4)
    sel_dtcluster_noise_2016 = np.logical_and(sel_dtcluster, np.logical_not(noise_2016))


########### SELECTION: JETS ############
    
    sel_jet = np.logical_and(T.array('jetPt') > 30, np.abs(T.array('jetEta')) < 2.4 )
    sel_jet = np.logical_and(T.array('jetTightPassId'), sel_jet)
            
########### SELECTION: NOISE IN DT ############
    
    spike = np.logical_and( T.array('nDTRechitsSector')[:,0,0,7]>50,  T.array('nDTRechitsSector')[:,0,0,7]+T.array('nDTRechitsSector')[:,0,0,8]+T.array('nDTRechitsSector')[:,0,0,9]>120)
    spike = np.logical_and(spike, T.array('nDTRechitsSector')[:,0,0,8]>25)
    spike = np.logical_and(spike, T.array('nDTRechitsSector')[:,0,0,9]>10)

    
    
########### SELECTION: EVENTS ############

    sel_ev[k] = T.array('METNoMuTrigger')
    sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('metEENoise') >= 200)
    sel_ev[k] = np.logical_and(sel_ev[k] , sel_jet.sum()>=1)
    sel_ev[k] = np.logical_and(sel_ev[k], (T.array('nDtRings')+T.array('nCscRings'))<10)
    sel_ev[k] = np.logical_and(sel_ev[k],T.array('Flag2_all'))
    sel_ev[k] = np.logical_and(sel_ev[k] , np.logical_not(spike))
########### BRANCHES ############
                            
    if category == 0:
        sel_ev[k]  = np.logical_and(sel_ev[k],sel_csccluster.sum()== 2)
        sel_ev[k]  = np.logical_and(sel_ev[k],sel_dtcluster.sum()== 0)
        cond[k] = deltaR(T.array('cscRechitCluster' + cluster_index + 'Eta')[sel_csccluster][sel_ev[k]][:,0], T.array('cscRechitCluster' + cluster_index + 'Phi')[sel_csccluster][sel_ev[k]][:,0],\
                        T.array('cscRechitCluster' + cluster_index + 'Eta')[sel_csccluster][sel_ev[k]][:,1], T.array('cscRechitCluster' + cluster_index + 'Phi')[sel_csccluster][sel_ev[k]][:,1])<2
        
       
        nhits1[k] =  T.array('cscRechitCluster' + cluster_index + 'Size')[sel_csccluster][sel_ev[k]][cond[k]][:,0]
        nhits2[k] =  T.array('cscRechitCluster' + cluster_index + 'Size')[sel_csccluster][sel_ev[k]][cond[k]][:,1]
#         deltaRCluster[k] = deltaR(T.array('cscRechitCluster' + cluster_index + 'Eta')[sel_csccluster][sel_ev[k]][:,0], T.array('cscRechitCluster' + cluster_index + 'Phi')[sel_csccluster][sel_ev[k]][:,0],\
#                         T.array('cscRechitCluster' + cluster_index + 'Eta')[sel_csccluster][sel_ev[k]][:,1], T.array('cscRechitCluster' + cluster_index + 'Phi')[sel_csccluster][sel_ev[k]][:,1])
    elif category == 1:
        
        
        # data
        if k == 'data':
            sel_ev[k]  = np.logical_and(sel_ev[k],sel_csccluster.sum()== 0)
            sel_ev[k]  = np.logical_and(sel_ev[k],sel_dtcluster.sum()== 2)
            nhits1[k] =  T.array('dtRechitClusterSize')[sel_dtcluster][sel_ev[k]][:,0]
            nhits2[k] =  T.array('dtRechitClusterSize')[sel_dtcluster][sel_ev[k]][:,1]
        else:
            # reweight to 1.5/fb, for events with dt noise
            sel_ev_dtnoise_2016[k]  = np.logical_and(sel_ev[k],sel_csccluster.sum() == 0)
            sel_ev_dtnoise_2016[k]  = np.logical_and(sel_ev_dtnoise_2016[k],sel_dtcluster_noise_2016.sum() == 2)

            if addNoiseFlag and not 'data' in k:
                nhits1_dtnoise_2016[k] =  (T.array('dtRechitClusterNoiseHit')+T.array('dtRechitClusterSize'))[sel_dtcluster_noise_2016][sel_ev_dtnoise_2016[k]][:,0]
                nhits2_dtnoise_2016[k] =  (T.array('dtRechitClusterNoiseHit')+T.array('dtRechitClusterSize'))[sel_dtcluster_noise_2016][sel_ev_dtnoise_2016[k]][:,1]
            else:
                nhits1_dtnoise_2016[k] =  T.array('dtRechitClusterSize')[sel_dtcluster_noise_2016][sel_ev_dtnoise_2016[k]][:,0]
                nhits2_dtnoise_2016[k] =  T.array('dtRechitClusterSize')[sel_dtcluster_noise_2016][sel_ev_dtnoise_2016[k]][:,1]

            # normal condition
            sel_ev[k]  = np.logical_and(sel_ev[k],sel_dtcluster.sum()== 2)
            sel_ev[k]  = np.logical_and(sel_ev[k],sel_csccluster.sum()== 0)
            if k == 'data':
                sel_ev[k] = np.logical_and()

            if addNoiseFlag and not 'data' in k:
                nhits1[k] =  (T.array('dtRechitClusterNoiseHit')+T.array('dtRechitClusterSize'))[sel_dtcluster][sel_ev[k]][:,0]
                nhits2[k] =  (T.array('dtRechitClusterNoiseHit')+T.array('dtRechitClusterSize'))[sel_dtcluster][sel_ev[k]][:,1]
            else:
                nhits1[k] =  T.array('dtRechitClusterSize')[sel_dtcluster][sel_ev[k]][:,0]
                nhits2[k] =  T.array('dtRechitClusterSize')[sel_dtcluster][sel_ev[k]][:,1]
        
#             nhits1[k] = np.hstack((nhits1[k], nhits1_dtnoise_2016[k]))
#             nhits2[k] = np.hstack((nhits2[k], nhits2_dtnoise_2016[k]))

    elif category == 2:
        # data
        if k == 'data':
            sel_ev[k]  = np.logical_and(sel_ev[k],sel_csccluster.sum()== 1)
            sel_ev[k]  = np.logical_and(sel_ev[k],sel_dtcluster.sum()== 1)
            cond[k] = deltaR(T.array('cscRechitCluster' + cluster_index + 'Eta')[sel_csccluster][sel_ev[k]][:,0], T.array('cscRechitCluster' + cluster_index + 'Phi')[sel_csccluster][sel_ev[k]][:,0],\
                            T.array('dtRechitCluster' + cluster_index + 'Eta')[sel_dtcluster][sel_ev[k]][:,0], T.array('dtRechitCluster' + cluster_index + 'Phi')[sel_dtcluster][sel_ev[k]][:,0])<2.5
            nhits1[k] =  T.array('dtRechitClusterSize')[sel_dtcluster][sel_ev[k]][cond[k]][:,0]
            nhits2[k] =  T.array('cscRechitClusterSize')[sel_csccluster][sel_ev[k]][cond[k]][:,0]
        else:
            sel_ev_dtnoise_2016[k]  = np.logical_and(sel_ev[k],sel_csccluster.sum() == 1)
            sel_ev_dtnoise_2016[k]  = np.logical_and(sel_ev_dtnoise_2016[k],sel_dtcluster_noise_2016.sum() == 1)

            sel_ev[k]  = np.logical_and(sel_ev[k],sel_csccluster.sum() == 1)
            sel_ev[k]  = np.logical_and(sel_ev[k],sel_dtcluster.sum() == 1)


            cond[k] = deltaR(T.array('cscRechitCluster' + cluster_index + 'Eta')[sel_csccluster][sel_ev[k]][:,0], T.array('cscRechitCluster' + cluster_index + 'Phi')[sel_csccluster][sel_ev[k]][:,0],\
                            T.array('dtRechitCluster' + cluster_index + 'Eta')[sel_dtcluster][sel_ev[k]][:,0], T.array('dtRechitCluster' + cluster_index + 'Phi')[sel_dtcluster][sel_ev[k]][:,0])<2.5
            if addNoiseFlag and not 'data' in k:
                nhits1[k] =  (T.array('dtRechitClusterNoiseHit')+T.array('dtRechitClusterSize'))[sel_dtcluster][sel_ev[k]][cond[k]][:,0]
            else:
                nhits1[k] =  T.array('dtRechitClusterSize')[sel_dtcluster][sel_ev[k]][cond[k]][:,0]
            nhits2[k] =  T.array('cscRechitCluster' + cluster_index + 'Size')[sel_csccluster][sel_ev[k]][cond[k]][:,0]

            # reweight to 1.5/fb, for events with dt noise
            cond_dtnoise_2016[k] = deltaR(T.array('cscRechitCluster' + cluster_index + 'Eta')[sel_csccluster][sel_ev_dtnoise_2016[k]][:,0], T.array('cscRechitCluster' + cluster_index + 'Phi')[sel_csccluster][sel_ev_dtnoise_2016[k]][:,0],\
                            T.array('dtRechitCluster' + cluster_index + 'Eta')[sel_dtcluster_noise_2016][sel_ev_dtnoise_2016[k]][:,0], T.array('dtRechitCluster' + cluster_index + 'Phi')[sel_dtcluster_noise_2016][sel_ev_dtnoise_2016[k]][:,0])<2.5
            if addNoiseFlag and not 'data' in k:
                nhits1_dtnoise_2016[k] =  (T.array('dtRechitClusterNoiseHit')+T.array('dtRechitClusterSize'))[sel_dtcluster_noise_2016][sel_ev_dtnoise_2016[k]][cond_dtnoise_2016[k]][:,0]
            else:
                nhits1_dtnoise_2016[k] =  T.array('dtRechitClusterSize')[sel_dtcluster_noise_2016][sel_ev_dtnoise_2016[k]][cond_dtnoise_2016[k]][:,0]
            nhits2_dtnoise_2016[k] =  T.array('cscRechitCluster' + cluster_index + 'Size')[sel_csccluster][sel_ev_dtnoise_2016[k]][cond_dtnoise_2016[k]][:,0]
#             nhits1[k] = np.hstack((nhits1[k], nhits1_dtnoise_2016[k]))
#             nhits2[k] = np.hstack((nhits2[k], nhits2_dtnoise_2016[k]))
    else:
        assert(False)
    if not k in 'data':
        if category == 0:
            weight[k] = (T.array('pileupWeight')*T.array('higgsPtWeight')*T.array('metSF'))*XSEC/NEVENTS
            weight[k][T.array('MC_condition')==2016] *= 35920 
            weight[k][T.array('MC_condition')==2017] *= 41530 
            weight[k][T.array('MC_condition')==2018] *= 59740 
            weight[k] = weight[k][sel_ev[k]][cond[k]]
        elif category == 1:
            
            weight[k] = (T.array('pileupWeight')*T.array('higgsPtWeight')*T.array('metSF'))*XSEC/NEVENTS
            weight[k][T.array('MC_condition')==2016] *= 35920 
            weight[k][T.array('MC_condition')==2017] *= 41530 
            weight[k][T.array('MC_condition')==2018] *= 59740 
            weight[k] = weight[k][sel_ev[k]]
            weight[k] *= (137-1.5)/137

#             weight_temp =  (T.array('pileupWeight')*T.array('higgsPtWeight')*T.array('metSF'))*XSEC/NEVENTS
#             weight_temp[T.array('MC_condition')==2016] *= 35920 
#             weight_temp[T.array('MC_condition')==2017] *= 41530 
#             weight_temp[T.array('MC_condition')==2018] *= 59740 
#             weight_temp = weight_temp[sel_ev_dtnoise_2016[k]]
#             weight[k] = np.hstack((weight[k], weight_temp * 1.5/137))
        else:
            weight[k] = (T.array('pileupWeight')*T.array('higgsPtWeight')*T.array('metSF'))*XSEC/NEVENTS
            weight[k][T.array('MC_condition')==2016] *= 35920 
            weight[k][T.array('MC_condition')==2017] *= 41530 
            weight[k][T.array('MC_condition')==2018] *= 59740 
            weight[k] = weight[k][sel_ev[k]][cond[k]]
            weight[k] *= (137-1.5)/137

#             weight_temp =  (T.array('pileupWeight')*T.array('higgsPtWeight')*T.array('metSF'))*XSEC/NEVENTS
#             weight_temp[T.array('MC_condition')==2016] *= 35920 
#             weight_temp[T.array('MC_condition')==2017] *= 41530 
#             weight_temp[T.array('MC_condition')==2018] *= 59740 
#             weight_temp = weight_temp[sel_ev_dtnoise_2016[k]][cond_dtnoise_2016[k]]
#             weight[k] = np.hstack((weight[k], weight_temp * 1.5/137))
            

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.15 µs


In [12]:
path_txt = '/storage/af/user/christiw/gpu/christiw/llp/delayed_jet_analyzer/plot_scripts/MuonSystem_Analysis/combination/2tag/hv_stat_unc/v2/'
with open(path_txt + 'category' + str(category)+'.txt', 'w') as f:
    f.write("sample name \t number of events in SR  \t relative uncertainty \n")
    for k in weight.keys():
#         if not k == 'HV_params_darkphoton_m_5_ctau_500mm_xiO_2p5_xiL_2p5':continue
        if category == 0:
            N_RECHIT_CUT_1 = 100
            N_RECHIT_CUT_2 = 100
        elif category == 1:
            N_RECHIT_CUT_1 = 80
            N_RECHIT_CUT_2 = 80
        else:
            N_RECHIT_CUT_1 = 80
            N_RECHIT_CUT_2 = 100
        cond_a = np.logical_and(nhits1[k]>=N_RECHIT_CUT_1, nhits2[k]>=N_RECHIT_CUT_2)#bin A
        if np.count_nonzero(cond_a) > 0 and np.sum(weight[k][cond_a]*weight[k][cond_a])**0.5/np.sum(weight[k][cond_a])<0.1:continue



        f.write(k + '\t' + str(len(weight[k][cond_a])) + '\t' + str(np.sum(weight[k][cond_a]**2)**0.5/np.sum(weight[k][cond_a])) + '\n')

#         print(k + '\t' + str(len(weight[k][cond_a])) + '\t' + str(np.sum(weight[k][cond_a]**2)**0.5/np.sum(weight[k][cond_a])) + '\n')


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in double_scalars


In [13]:
print('category', category)
k = 'data'
k = 'HV_params_darkphoton_m_5_ctau_500mm_xiO_2p5_xiL_2p5'
if category == 0:
    N_RECHIT_CUT_1 = 100
    N_RECHIT_CUT_2 = 100
elif category == 1:
    N_RECHIT_CUT_1 = 80
    N_RECHIT_CUT_2 = 80
else:
    N_RECHIT_CUT_1 = 80
    N_RECHIT_CUT_2 = 100
    

cond_a = np.logical_and(nhits1[k]>=N_RECHIT_CUT_1, nhits2[k]>=N_RECHIT_CUT_2)#bin A
cond_b =  np.logical_and(nhits1[k]<N_RECHIT_CUT_1, nhits2[k]>=N_RECHIT_CUT_2)
if category == 2:cond_d = np.logical_and(nhits1[k]>=N_RECHIT_CUT_1, nhits2[k]<N_RECHIT_CUT_2)#bin A
else:cond_b = np.logical_or(cond_b, np.logical_and(nhits1[k]>=N_RECHIT_CUT_1, nhits2[k]<N_RECHIT_CUT_2))#bin A
cond_c = np.logical_and(nhits1[k]<N_RECHIT_CUT_1, nhits2[k]<N_RECHIT_CUT_2)#bin 
# cond = np.abs(deltaRCluster[k])<2000
# cond_a = np.logical_and(cond, cond_a)
# cond_b = np.logical_and(cond, cond_b)
# cond_c = np.logical_and(cond, cond_c)
# if category == 2:cond_d = np.logical_and(cond, cond_d)

if category == 2: 
    print(np.count_nonzero(cond_b), np.count_nonzero(cond_c), np.count_nonzero(cond_d))

    print(np.sum(weight[k][cond_a]), np.sum(weight[k][cond_b]), np.sum(weight[k][cond_c]), np.sum(weight[k][cond_d]))

else:
    print(np.count_nonzero(cond_b), np.count_nonzero(cond_c), np.count_nonzero(cond_a))
    print(np.sum(weight[k][cond_b]), np.sum(weight[k][cond_c]), np.sum(weight[k][cond_a]))

category 0
18 7 10
17.174353249225614 8.254761341781615 9.798154972000123


In [60]:
BR = 0.01
file1 = open("category"+str(category)+".txt","w")
file1.write("sample nane \t relative uncertainty \t signal yield (BR = 1) \t Nevents \n")
for N_RECHIT_CUT_1 in np.arange(60,220,1):
    for N_RECHIT_CUT_2 in np.arange(60,220,1):
        if category < 2 and not N_RECHIT_CUT_1 == N_RECHIT_CUT_2:continue
        if category == 0 and not N_RECHIT_CUT_1 == N_RECHIT_CUT_2 == 100:continue
        if category == 1 and not N_RECHIT_CUT_1 == 80:continue
        if category == 2 and (not N_RECHIT_CUT_1 == 80 or not N_RECHIT_CUT_2 == 100):continue
        print(category, N_RECHIT_CUT_1, N_RECHIT_CUT_2)
      
        for k in weight.keys():
            if k == 'data':continue
            if np.count_nonzero(sel_ev[k]) == 0: continue
            # signal region
            cond_d = np.logical_and(nhits1[k]>=N_RECHIT_CUT_1, nhits2[k]>=N_RECHIT_CUT_2)#bin A
            if np.count_nonzero(cond_d) == 0: continue
            if category == 0:w = weight[k][cond_d]
            else: w = weight[k][cond_d]
#             print(k, np.sum(w))

#             if np.sum(weight[k][cond_d]*weight[k][cond_d])**0.5/np.sum(weight[k][cond_d])>0.2:
#                 print(k, '\t',np.sum(weight[k][cond_d]*weight[k][cond_d])**0.5/np.sum(weight[k][cond_d]))
#     , weight[k][cond_d].min(), weight[k][cond_d].max(), np.mean(weight[k][cond_d]))
            if np.sum(weight[k][cond_d]*weight[k][cond_d])**0.5/np.sum(weight[k][cond_d])<0.1:continue
            file1.write(k+'\t'+str(np.sum(weight[k][cond_d]*weight[k][cond_d])**0.5/np.sum(weight[k][cond_d]))+'\t'+str(np.sum(weight[k][cond_d]))+'\t'+str(NEvents[k])+'\n')
file1.close() #to change file access modes
#             print(k, '\t', NEvents[k])
#             print(k, '\t',np.sum(weight[k][cond_d]), '\t',np.sum(weight[k][cond_d]*weight[k][cond_d])**0.5, np.sum(weight[k][cond_d]*weight[k][cond_d])**0.5/np.sum(weight[k][cond_d]))

1 80 80


IndexError: boolean index did not match indexed array along dimension 0; dimension is 809 but corresponding boolean dimension is 406

# signal yield summary table

In [61]:
ctaus = [ '100', '1000','10000','100000' ]


BR = 0.01

for N_RECHIT_CUT_1 in np.arange(60,220,1):
    for N_RECHIT_CUT_2 in np.arange(60,220,1):
        if category < 2 and not N_RECHIT_CUT_1 == N_RECHIT_CUT_2:continue
        if category == 0 and not N_RECHIT_CUT_1 == N_RECHIT_CUT_2 == 100:continue
        if category == 1 and not N_RECHIT_CUT_1 == 80:continue
        if category == 2 and (not N_RECHIT_CUT_1 == 80 or not N_RECHIT_CUT_2 == 100):continue
        print(category, N_RECHIT_CUT_1, N_RECHIT_CUT_2)
        
        total_sig = 0
        for m in mass:
    #         if not m == 55:continue
            signal_rate = []
            unc_rate = []
            signal_unc = []
            for ct in ctaus:   
    #             if not ct == '1000':continue
                signal = 0
                sig_unc = 0
                shape_unc_temp = 0
                ctf = int(ct)
                if ctf < OLD_CTAU[0]:
                    old_ctau_temp = np.array([OLD_CTAU[0]])


                weight_sum = 0
                weight_len = 0
                for j,ct0 in enumerate(old_ctau_temp):
                    'ggH'ds = ['ggH', 'VBFH_H','WH','ZH','ttH_H','ggZH']
                    'ggH'ds = ['ggH']
                    for p in 'ggH'ds:

#                         k = 'MC_'+p+'_'+str(m)+'_'+str(ct0)+''

                        k = 'MC_ggH_STodd_ms3p0_100'
                        k = 'MC_ggH_STodd_ms15_1000'
                        if np.count_nonzero(sel_ev[k]) == 0: continue

        
                        # signal region
                        cond_d = np.logical_and(nhits1[k]>=N_RECHIT_CUT_1, nhits2[k]>=N_RECHIT_CUT_2)#bin A

                        signal += np.sum(w[cond_d])
                        sig_unc +=np.sum(w[cond_d]*w[cond_d])


                signal_rate.append(signal)
                signal_unc.append(sig_unc**0.5)
                unc_rate.append(shape_unc_temp)
            signal_rate = np.array(signal_rate)
            unc_rate = np.array(unc_rate)
            signal_unc = np.array(signal_unc)
#             print(m,":", [num for num in signal_rate*BR*signal_correction])
#             print(m,":", [num for num in signal_unc*BR*signal_correction])
            print(m, '\t', '\t'.join(map(str, [round(num,5) for num in signal_rate*BR])))
            total_sig += np.sum(signal_rate*BR)
        print(total_sig)
#             print(m, '\t', '\t'.join(map(str, [round(num,5) for num in signal_rate*BR*signal_correction])))
#             print(m, 'GeV &', ' & '.join(map(str, [round(num,1) for num in signal_rate*BR*signal_correction])), '\\\\')


SyntaxError: invalid syntax (<ipython-input-61-a58a9750088c>, line 33)

category 1
1 3


In [48]:
# datacard_version  = 'v1' # ABCD for all categories
datacard_version  = 'v2' # ABCD for CSC-DT and 3 bins for CSC-CSC and DT-DT
datacard_version  = 'v3' # signal uncertainties and corrections applied
datacard_version  = 'v4' # global muon veto for csc and loose id muon veto for dt
datacard_version  = 'v5' # global muon veto + eta cut for csc and loose id muon veto for dt
datacard_version  = 'v6' # global muon veto for csc and loose id muon veto + station veto +tight ID jet veto for dt

datacard_version  = 'v7' # deltaEta cut 2.5 (DT-CSC) and 1.5 (CSC-CSC)
datacard_version  = 'v8' # deltaEta cut 2 (DT-CSC) and 1.5 (CSC-CSC)

datacard_version  = 'v9' # deltaR cut for CSC and DT
datacard_version  = 'v10' # change ABCD rateParam to the simple case
datacard_version  = 'v11' # change ABCD rateParam to the simple case


# signal uncertainties

In [49]:
cscCluster_mc = {}
cscCluster_data = {}
time_spread_mc = {}
time_spread_data = {}
dtCluster = {}
higgsPtWeight = {}
JES = {}
pileup = {}
# ctau_reweight = {}
mc_stats = {}
scaling = {}
lumi = {}
time_spread = {}
time = {}
theory = {}
pdf = {}
readout = {}
ggH_higgsPt , VBFH_higgsPt , WH_higgsPt , ZH_higgsPt , ttH_higgsPt , ggZH_higgsPt =({} for i in range(6)) 
ggH_xsec , VBFH_xsec , WH_xsec , ZH_xsec , ttH_xsec , ggZH_xsec =({} for i in range(6)) 
ggH_pdf , VBFH_pdf , WH_pdf , ZH_pdf , ttH_pdf , ggZH_pdf =({} for i in range(6)) 
Nrechit_weight_file = '/storage/user/christiw/login-1/christiw/LLP/CMSSW_9_4_4/src/llp_analyzer/data/Uncertainties/Nrechit_weight.txt'

cut_based_unc = [0.0921, 0.0852] # muon Eta reweighted
clustering_unc = [0.1506, 0.1047]
if category == 2 or datacard_version == 'v1': bins = 4
else: bins = 3
print('category: '+ str(category))
print('bins: '+ str(bins))

for x in scale:
    for p in portal.keys():
        for m in portal[p]:
            for ct in old_ctau:
                k = 'HV_params_'+p+'_m_'+str(m) + '_ctau_'+str(ct)+'mm_'+x
                key = 'HV_params_'+p+'_m_'+str(m) + '_ctau_'+str(ct)+'mm_'+x


                # CSC hits
                if category == 0:
                    cscCluster_mc[key] = [0.02] * bins
                    cscCluster_data[key] = [0.012] * bins
                    time_spread_mc[key] = [0.037*2] * bins
                    time_spread_data[key] = [0.039*2] * bins
                    dtCluster[key] = [0.0] * bins
                    JES[key] = [0.0351,0.0573, 0.0470] 
                    pileup[key] = [0.0102,0.0144,0.0183]

                elif category == 1:
                    cscCluster_mc[key] = [0.0] * bins
                    cscCluster_data[key] = [0.0] * bins
                    time_spread_mc[key] = [0.0] * bins
                    time_spread_data[key] = [0.0] * bins
                    dtCluster[key] = [0.032] * bins
                    JES[key] = [0.027, 0.036,0.079]
                    pileup[key] = [0.013, 0.0152, 0.0783]
                else: 
                    cscCluster_mc[key] = [0.013] * bins
                    cscCluster_data[key] = [0.017] * bins
                    time_spread_mc[key] = [0.037] * bins
                    time_spread_data[key] = [0.039] * bins
                    dtCluster[key] = [0.011] * bins
                    JES[key] = [0.0332, 0.0306, 0.1211,0.039]
                    pileup[key] = [0.011,0.019, 0.018,0.025]


                lumi[key] = [0.018]*bins



                ggH_higgsPt[key], VBFH_higgsPt[key], WH_higgsPt[key] , ZH_higgsPt[key] , ttH_higgsPt[key] , ggZH_higgsPt[key] = ([0.0]*bins*2,)*6
                ggH_xsec[key], VBFH_xsec[key], WH_xsec[key] , ZH_xsec[key] , ttH_xsec[key] , ggZH_xsec[key] = ([0.0]*bins*2,)*6
                ggH_pdf[key] , VBFH_pdf[key] , WH_pdf[key] , ZH_pdf[key] , ttH_pdf[key] , ggZH_pdf[key], scaling[key] = ([0.0]*bins,)*7
                
    #                 if category == 2 or datacard_version == 'v1': ggH_higgsPt[key] = [0.205, 0.205,0.207,0.208, 0.133,0.133,0.134,0.134] #down*4/up*4
    #                 else:  ggH_higgsPt[key] = [0.205, 0.205,0.207, 0.133,0.133,0.134] #down*4/up*4

                if category == 0:ggH_higgsPt[key] = [0.2066,0.2069, 0.2069,0.1339,0.134,0.1341] #down*3/up*3
                elif category == 1:ggH_higgsPt[key] = [0.2074,0.2077, 0.2061,0.1342,0.1343,0.1336] #down*3/up*3
                else:ggH_higgsPt[key] = [0.2071, 0.2066, 0.2084, 0.2076, 0.134,0.134,0.1345,0.1343] #down*4/up*4
                ggH_xsec[key] = [0.067]*bins+[0.046]*bins #down/up
                ggH_pdf[key] = [0.032]*bins



sig_eff = [lumi, cscCluster_mc, cscCluster_data, time_spread_mc, time_spread_data, dtCluster, JES,pileup, \
           ggH_higgsPt, VBFH_higgsPt, WH_higgsPt, ZH_higgsPt, ttH_higgsPt, ggZH_higgsPt,  \
           ggH_xsec, VBFH_xsec, WH_xsec, ZH_xsec, ttH_xsec, ggZH_xsec, \
           ggH_pdf,VBFH_pdf, WH_pdf, ZH_pdf, ttH_pdf, ggZH_pdf,\
           scaling, mc_stats]
sig_eff_name = ['lumi','cscCluster_nhits_mc', 'cscCluster_nhits_data', 'csc_timeSpread_mc', 'csc_timeSpread_data', 'dtCluster_nhits', 'JES','pileup', \
                'ggH_higgsPt', 'VBFH_higgsPt', 'WH_higgsPt', 'ZH_higgsPt', 'ttH_higgsPt', 'ggZH_higgsPt',\
                'ggH_xsec', 'VBFH_xsec', 'WH_xsec', 'ZH_xsec', 'ttH_xsec', 'ggZH_xsec', \
                'ggH_pdf','VBFH_pdf', 'WH_pdf', 'ZH_pdf', 'ttH_pdf', 'ggZH_pdf',\
                'ggZH_reweight','category'+str(category)+'mc_stats']


# csc: jet, muon, time, time spread, clustereff_unc, readout
# theory uncertainty for now
assert(len(sig_eff)==len(sig_eff_name))

category: 1
bins: 3


# make datacard (ABCD)

In [50]:

UNBLIND = 1
# 0 use exponential fit, 1 unblind ABC, 2 unblind ABCD
N_RECHIT_CSC = 100
N_RECHIT_DT = 80


#####
# a = c*c1*c2
# b = c1* c
# c = c
# d = c2*c
# start adding signal
#####
outDataCardsDir = '/storage/af/user/christiw/login-1/christiw/LLP/CMSSW_10_2_13/src/HiggsAnalysis/MuonSystemLimit/combine/datacards_2tag/'+\
ntupler_version+analyzer_version+'/'
outDataCardsDir += datacard_version +'/'
if UNBLIND == 0: outDataCardsDir += 'blind/'
elif UNBLIND == 1: outDataCardsDir += 'unblindABC/'
else:outDataCardsDir += 'unblind/'
outDataCardsDir += 'category'+str(category)+'/'
if not os.path.isdir(outDataCardsDir):os.makedirs(outDataCardsDir)
print(outDataCardsDir)
if category == 2 or datacard_version == 'v1': bins = 4
else: bins = 3


#############
# BACKGROUND
#############

bkg_unc = [] #ABCD closure, sum of stats from two VR
bkg_unc_name = []
if UNBLIND == 0:
    ##### v138 #####
    if 'v138' in analyzer_version:
        if category == 0:bkg_rate = np.array([0.062*0.073,0.073,1,0.062])*59 #2csc
        elif category == 1:bkg_rate = np.array([0.02*0.01,0.02,1,0.01])*44 # dt
        elif category == 2:bkg_rate = np.array([0.07*0.15,0.07,1,0.15])*66 #csc+dt
    elif 'v141' in analyzer_version:
        if datacard_version == 'v1':
            if category == 0:bkg_rate = np.array([0.03*0.03,0.03,1 ,0.03])*18 #2csc
            elif category == 1:bkg_rate = np.array([0.03*0.03,0.03,1, 0.03])*19 # dt
            elif category == 2:bkg_rate = np.array([0.009*0.025,0.009,1,0.025])*35 #csc+dt
        elif datacard_version == 'v2':
            if category == 0:bkg_rate = np.array([0.03*0.03,0.03*2,1])*18 #2csc
            elif category == 1:bkg_rate = np.array([0.03*0.03,0.03*2,1])*19 # dt
            elif category == 2:bkg_rate = np.array([0.009*0.025,0.009,1,0.025])*35 #csc+dt
    elif 'v146' in analyzer_version: #with dtcosmic muon veto
        if datacard_version == 'v2':
            if category == 0:bkg_rate = np.array([0.09, 2.5, 18])
            elif category == 1:bkg_rate = np.array([0.02, 1.2, 17.9])
            elif category == 2:bkg_rate = np.array([0.02, 0.7, 34.2, 0.9])
elif UNBLIND == 1:
    k = 'data'
    if category <= 1:
        if category == 0: N_RECHIT_CUT = N_RECHIT_CSC
        else: N_RECHIT_CUT = N_RECHIT_DT
#             cond_a = np.logical_and(nhits1[k]>=N_RECHIT_CUT, nhits2[k]>=N_RECHIT_CUT)#bin A
        b = np.logical_and(nhits1[k]>=N_RECHIT_CUT, nhits2[k]<N_RECHIT_CUT)#bin A
        b = np.count_nonzero(np.logical_or(b, np.logical_and(nhits1[k]<N_RECHIT_CUT, nhits2[k]>=N_RECHIT_CUT)))
        c = np.count_nonzero(np.logical_and(nhits1[k]<N_RECHIT_CUT, nhits2[k]<N_RECHIT_CUT))
        bkg_rate = np.array([b**2/4.0/c, b, c])

    else:
        N_RECHIT_CUT_1 = N_RECHIT_DT
        N_RECHIT_CUT_2 = N_RECHIT_CSC
        cond_a = np.logical_and(nhits1[k]>=N_RECHIT_CUT_1, nhits2[k]>=N_RECHIT_CUT_2)#bin A
        b = np.count_nonzero(np.logical_and(nhits1[k]<N_RECHIT_CUT_1, nhits2[k]>=N_RECHIT_CUT_2))
        d = np.count_nonzero(np.logical_and(nhits1[k]>=N_RECHIT_CUT_1, nhits2[k]<N_RECHIT_CUT_2))
        c = np.count_nonzero(np.logical_and(nhits1[k]<N_RECHIT_CUT_1, nhits2[k]<N_RECHIT_CUT_2))
        bkg_rate = np.array([b*d/c, b, c, d])
    print('category: '+str(category))
    print('background: ', bkg_rate)

else: assert(False)
observation= bkg_rate #blinded observation
print(observation, bkg_rate)
#############
# SIGNAL
#############


sig_norm = []
for k in weight.keys():
    modelName = k + '_nCsc'+str(N_RECHIT_CSC)+'_nDt'+str(N_RECHIT_DT)

    signal_rate = {}
    mc_stat_unc = {}
    gmn = {}
    sig_unc = {}


    signal_rate['ggH'] = np.zeros((bins,))
    mc_stat_unc['ggH'] = np.zeros((bins,))
    gmn['ggH'] = np.zeros((bins,))
    sig_unc['ggH'] = []

    w = weight[k]
    weight_cond = np.ones(nhits2[k].shape)
    if category == 2:
        a = np.logical_and(np.logical_and(nhits1[k]>=N_RECHIT_DT, nhits2[k]>=N_RECHIT_CSC), weight_cond)
        signal_rate['ggH'][0]+=np.sum(w[a])
        mc_stat_unc['ggH'][0]+=np.sum(w[a]**2)
        gmn['ggH'][0] += len(w[a])
        b = np.logical_and(np.logical_and(nhits1[k]<N_RECHIT_DT, nhits2[k]>=N_RECHIT_CSC), weight_cond)
        signal_rate['ggH'][1]+=np.sum(w[b])
        mc_stat_unc['ggH'][1]+=np.sum(w[b]**2)
        gmn['ggH'][1] += len(w[b])
        c = np.logical_and(np.logical_and(nhits1[k]<N_RECHIT_DT, nhits2[k]<N_RECHIT_CSC), weight_cond)
        signal_rate['ggH'][2]+=np.sum(w[c])
        mc_stat_unc['ggH'][2]+=np.sum(w[c]**2)
        gmn['ggH'][2] += len(w[c])
        d = np.logical_and(np.logical_and(nhits1[k]>=N_RECHIT_DT, nhits2[k]<N_RECHIT_CSC), weight_cond)
        signal_rate['ggH'][3]+=np.sum(w[d])
        mc_stat_unc['ggH'][3]+=np.sum(w[d]**2)
        gmn['ggH'][3] += len(w[d])
    elif datacard_version == 'v1':
        if category == 0:N_RECHIT_CUT = N_RECHIT_CSC
        else: N_RECHIT_CUT = N_RECHIT_DT

        a = np.logical_and(np.logical_and(nhits1[k]>=N_RECHIT_CUT, nhits2[k]>=N_RECHIT_CUT), weight_cond)
        signal_rate['ggH'][0]+=np.sum(w[a])
        mc_stat_unc['ggH'][0]+=np.sum(w[a]**2)
        gmn['ggH'][0] += len(w[a])
        b = np.logical_and(np.logical_and(nhits1[k]<N_RECHIT_CUT, nhits2[k]>=N_RECHIT_CUT), weight_cond)
        signal_rate['ggH'][1]+=np.sum(w[b])
        mc_stat_unc['ggH'][1]+=np.sum(w[b]**2)
        gmn['ggH'][1] += len(w[b])
        c = np.logical_and(np.logical_and(nhits1[k]<N_RECHIT_CUT, nhits2[k]<N_RECHIT_CUT), weight_cond)
        signal_rate['ggH'][2]+=np.sum(w[c])
        mc_stat_unc['ggH'][2]+=np.sum(w[c]**2)
        gmn['ggH'][2] += len(w[c])
        d = np.logical_and(np.logical_and(nhits1[k]>=N_RECHIT_CUT, nhits2[k]<N_RECHIT_CUT), weight_cond)
        signal_rate['ggH'][3]+=np.sum(w[d])
        mc_stat_unc['ggH'][3]+=np.sum(w[d]**2)
        gmn['ggH'][3] += len(w[d])

    else:
        ####
        # a is signal region, b has one cluster pass Nrechit cut, C has no cluster passing Nrechit cut
        ####
        if category == 0:N_RECHIT_CUT = N_RECHIT_CSC
        else: N_RECHIT_CUT = N_RECHIT_DT
        a = np.logical_and(np.logical_and(nhits1[k]>=N_RECHIT_CUT, nhits2[k]>=N_RECHIT_CUT), weight_cond)
        signal_rate['ggH'][0]+=np.sum(w[a])
        mc_stat_unc['ggH'][0]+=np.sum(w[a]**2)
        gmn['ggH'][0] += len(w[a])

        b = np.logical_and(np.logical_and(nhits1[k]<N_RECHIT_CUT, nhits2[k]>=N_RECHIT_CUT), weight_cond)
        d = np.logical_and(np.logical_and(nhits1[k]>=N_RECHIT_CUT, nhits2[k]<N_RECHIT_CUT), weight_cond)
        b = np.logical_or(b, d)
        signal_rate['ggH'][1]+=np.sum(w[b])
        mc_stat_unc['ggH'][1]+=np.sum(w[b]**2)
        gmn['ggH'][1] += len(w[b])

        c = np.logical_and(np.logical_and(nhits1[k]<N_RECHIT_CUT, nhits2[k]<N_RECHIT_CUT), weight_cond)
        signal_rate['ggH'][2]+=np.sum(w[c])
        mc_stat_unc['ggH'][2]+=np.sum(w[c]**2)
        gmn['ggH'][2] += len(w[c])

    for j in range(bins):
        if signal_rate['ggH'][j]<0.0:
            signal_rate['ggH'][j]=0.0
            mc_stat_unc['ggH'][j]=0.0
            gmn['ggH'][j] = 0.0



    mc_stat_unc['ggH'] = np.nan_to_num(signal_rate['ggH']/np.sqrt(mc_stat_unc['ggH']))**2 #gmn
    mc_stat_unc['ggH'][mc_stat_unc['ggH'] == np.inf] = 0.0

    mc_stat_unc['ggH'] = list(mc_stat_unc['ggH'])
    mc_stat_unc['ggH'] = list(gmn['ggH'])
    if p == 'others': k = k.replace("ggH", "others")
    for j, ele in enumerate(sig_eff):# go through each uncertainty
        if j == len(sig_eff)-1: # if mc_stats
            sig_unc['ggH'].append(mc_stat_unc['ggH'])
        else:
            sig_unc['ggH'].append(ele[k])


    norm = np.sum(signal_rate['ggH'])/4
    #only used for calculation, the signal yield is not changed in datacard


    if category == 2 or datacard_version == 'v1': make_datacard_2tag(outDataCardsDir, modelName, signal_rate, norm, bkg_rate, observation, \
                      bkg_unc, bkg_unc_name, sig_unc, sig_eff_name, 'a', 'category'+str(category)+'_')
    else: make_datacard_2tag_3bins(outDataCardsDir, modelName, signal_rate, norm, bkg_rate, observation, \
                          bkg_unc, bkg_unc_name, sig_unc, sig_eff_name, 'a', 'category'+str(category)+'_')


/storage/af/user/christiw/login-1/christiw/LLP/CMSSW_10_2_13/src/HiggsAnalysis/MuonSystemLimit/combine/datacards_2tag/V1p17/v2/v163//v11/unblindABC/category1/
category: 1
background:  [0.08333333 1.         3.        ]
[0.08333333 1.         3.        ] [0.08333333 1.         3.        ]


# unblind

In [26]:
import math

In [13]:
k = 'data_intime_sr'

n_ev = 5000
var = np.abs(dphiMet_cluster[k])
DPHI_CUT = 0.75
print("Nrechits, A, B, C, pred, D")
for N_RECHIT_CUT in np.arange(60,240,10):
    a = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=N_RECHIT_CUT, var>=DPHI_CUT))
    b = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]<N_RECHIT_CUT, var>=DPHI_CUT))
    c = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]<N_RECHIT_CUT, var<DPHI_CUT))
    d = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=N_RECHIT_CUT, var<DPHI_CUT))
    pred = c/b*a
    unc_pred = (1./c + 1./b + 1./a)**0.5*(c/b*a)

    print(N_RECHIT_CUT, '\t',a,'\t',b,'\t',c,'\t',d,'\t', round(c/b*a, 2), '\t',\
          round( (1./c + 1./b + 1./a)**0.5*(c/b*a), 2))


NameError: name 'dphiMet_cluster' is not defined